In [1]:
from music21 import *
from collections import Counter, defaultdict

import collections
import numpy as np
import pandas as pd
import gzip
import random
import initialisation
import modelTrainer
import musicPlayer
import simulation
import fitness
import collections
import importlib
import music21
import numpy as np
import pandas as pd
import individual
import duration
import constants
import util
import random
import nltk
import fitness
import re

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
importlib.reload(initialisation)
importlib.reload(modelTrainer)
importlib.reload(musicPlayer)
importlib.reload(simulation)
importlib.reload(fitness)
importlib.reload(music21)
importlib.reload(constants)
importlib.reload(util)
importlib.reload(individual)
importlib.reload(fitness)

<module 'fitness' from 'B:\\School\\Master\\Thesis\\MasterThesis\\EvoMusicCompanion\\fitness.py'>

In [2]:
wiki_corpus = corpus.corpora.LocalCorpus('wiki')
curr_corpus = wiki_corpus.metadataBundle[0:20]

scores = []
for c in curr_corpus:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(score)

In [86]:
sections = [('intro',2), ('verse', 8), ('chorus', 4), ('verse', 8), ('chorus', 4)]


In [4]:
class Individual:
    def __init__(self, sections, fitness):
        self.sections = sections
        self.fitness = fitness
class Section:
    def __init__(self, name, measures):
        self.name = name
        self.measures = measures
class Measure:
    def __init__(self, notes, chord):
        self.notes = notes
        self.chord = chord
class Note: 
    def __init__(self, pitch, duration):
        self.pitch = pitch
        self.duration = duration


In [108]:
len(p.sections['intro'][0].notes)


8

In [116]:
population = init_population_with_sections(sections)
p = population[0]
score = stream.Score(id='mainScore')
part = stream.Part(id='part0')
counter = 0
for s in p.sections:
    for measure in range(len(p.sections[s])):
        m = stream.Measure(counter)
        measure = p.sections[s][measure]
        counter += 1
        for j in measure.notes:
            if j.pitch == 'REST':
                n = note.Rest()
                n.duration = music21.duration.Duration(quarterLength=j.duration.duration_value/0.25)
            else:
                n = note.Note(j.pitch)
                n.duration = music21.duration.Duration(quarterLength=j.duration.duration_value/0.25)
            m.append(n)
        part.append(m)
score.append(part)

In [117]:
score.show('musicxml')

In [58]:
"test" + "1"

'test1'

In [115]:
def init_population_with_sections(sections):
    population = []
    for i in range(10):
        # For each section 
        sec = {}
        for s in range(len(sections)):
            section_name = sections[s][0]
            section_num_of_measures = sections[s][1]
            measures = []
            
             # If the section already exists, we repeat it but prefix it with a number
            counter = 2
            if section_name in sec:
                section_name_prefixed = section_name + str(counter)
                sec[section_name_prefixed] = sec[section_name]
                continue
                
            # Generate measures
            for j in range(0, section_num_of_measures):
                # 1. Assign duration
                measure_duration = get_duration_for_measure()
                # 2. Get notes with pitches by duration array
                new_notes = get_notes_by_duration(measure_duration)
                # 3. Add to measure
                m = Measure(new_notes, None)
                measures.append(m)

            # 4. Add measure to section    
            new_section = Section(measures, section_name)
            sec[section_name] = measures

        new_individual = Individual(sec, 0)
        population.append(new_individual)
        
    return population

In [6]:
duration_matrix = modelTrainer.train_duration_matrix(scores)

In [7]:
pitch_matrix = modelTrainer.train_pitch_matrix(scores)

In [85]:
d = duration.Duration('dotted-quarter', None)
note.Note('C4', d)

TypeError: __init__() takes from 1 to 2 positional arguments but 3 were given

In [130]:
durations = get_duration_for_measure()
notes = get_notes_by_duration(durations)
for n in notes:
    print(n.pitch)

E4
F4
F4
E4


In [8]:
def get_duration_for_measure():
    dur_counter = 0.0
    durations = []
    while(dur_counter < 1.0):
        # 1. Get initial duration
        if dur_counter == 0.0:
            new_d = initialisation.get_random_transition(duration_matrix, None)
            new_d = duration.Duration(new_d, None)
            dur_counter += new_d.duration_value
            durations.append(new_d)
            continue
            
        # 2. Keep generating duration until we reach maximum duration
        new_d = initialisation.get_random_transition(duration_matrix, durations[-1].duration_name)
        new_d = duration.Duration(new_d, None)
        
        # 3. If adding next one exceeds: Either lengthen the last note or add a rest
        (exceeds, remaining) = exceeds_duration(new_d.duration_value, dur_counter)
        if exceeds and remaining > 0:
            # Lengthen last note if duration is < 16th note
            if remaining < 0.125:
                durations[-1].duration_value += remaining
                break
            # Set duration to remaining duration            
            else:
                new_d.duration_value += remaining
        dur_counter += new_d.duration_value
        
        # Append to durations
        durations.append(new_d)
        
    return durations

def exceeds_duration(new_duration, curr_duration):
    # Check how much duration exceeds when adding the new duration
    total_duration = curr_duration + new_duration

    max_duration_left = 1.0 - curr_duration
    # If this is more than 1.0, we can decrease the length of this duration or we have to remove it
    if total_duration > 1.0:
        # If there is still room for another note, change it to that duration
        if max_duration_left > 0.0:
            return True, max_duration_left
        # Else we do not want to add this note
        else:
            return True, 0.0
    return False, 0.0

def to_float_duration(duration_name):
    if duration_name == 'quarter':
        return 0.25
    elif duration_name == 'eighth':
        return 0.125
    elif duration_name == 'whole':
        return 1.0
    elif duration_name == 'half':
        return 0.5
    elif duration_name == '16th':
        return 0.0625
    elif duration_name == '32nd':
        return 0.03125
    elif duration_name == '64th':
        return 0.015625
    else:
        return 0.015625

In [9]:
def get_notes_by_duration(durations):
    notes = []
    for i in range(len(durations)):
        curr_dur = durations[i]
        if i == 0:
            curr_pitch = initialisation.get_random_transition(pitch_matrix, None)
            n = Note(curr_pitch, curr_dur)
            notes.append(n)
            continue
        curr_pitch = initialisation.get_random_transition(pitch_matrix, notes[-1].pitch)
        n = Note(curr_pitch, curr_dur)
        notes.append(n)
    return notes
            